In [1]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), download_data=False, verbose=False)

Setting up Sandbox...
Done!


In [2]:
x = th.tensor([1,2,3,4]).send(bob)

In [3]:
sy.serde._apply_compress_scheme = sy.serde.apply_no_compression

In [4]:
b = sy.serde.serialize(bob, force_no_serialization=False, force_full=True)

In [5]:
b

b'(\x92\x11\x92\x92\x12\x91\xa3bob\x92\x05\x91\x92\xcf\x00\x00\x00\x0c\x8d\xff\xac\xd6\x92\x00\x96\xcf\x00\x00\x00\x0c\x8d\xff\xac\xd6\xda\x01r\x80\x02\x8a\nl\xfc\x9cF\xf9 j\xa8P\x19.\x80\x02M\xe9\x03.\x80\x02}q\x00(X\x10\x00\x00\x00protocol_versionq\x01M\xe9\x03X\r\x00\x00\x00little_endianq\x02\x88X\n\x00\x00\x00type_sizesq\x03}q\x04(X\x05\x00\x00\x00shortq\x05K\x02X\x03\x00\x00\x00intq\x06K\x04X\x04\x00\x00\x00longq\x07K\x04uu.\x80\x02ctorch._utils\n_rebuild_tensor_v2\nq\x00((X\x07\x00\x00\x00storageq\x01ctorch\nLongStorage\nq\x02X\x0f\x00\x00\x00140537124320048q\x03X\x03\x00\x00\x00cpuq\x04K\x04Ntq\x05QK\x00K\x04\x85q\x06K\x01\x85q\x07\x89ccollections\nOrderedDict\nq\x08)Rq\ttq\nRq\x0b.\x80\x02]q\x00X\x0f\x00\x00\x00140537124320048q\x01a.\x04\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xc0\xc0\xc0\xc0'

In [6]:
m = sy.serde.deserialize(b)